# Memory Bandwidth-Bound Problems and GPUs

GPUs are generally thought of as an accelerator for compute-bound applications, but they also offer large performance boosts for problems that are memory bandwidth-bound: 

<img src="https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/memory-bandwidth.png" alt="Drawing" style="width: 500px;"/>

The peak theoretical bandwidth of a NVIDIA grapchis card compared to an Intel Xeon CPU are shown in the table below.

| NVIDIA Pascal Titan X  |  Intel Xeon E5-2623 v4 |
|:-:|:-:|
|  480 GB/s | 68 GB/s  

## Vector Addition Example
 
A simple element-wise vector addition is a good example of a common memory-bound function:

```C
void vec_add(float * A, float * B, float * C, int N)
{
   for (int i=0; i<N; i++)
      C[i] = A[i] + B[i];
}```
This function consists of a total of N floating-point operations and 3*N memory access operations (2 reads and 1 write per array element), so memory accesses are likely to be the bottleneck, rather than mathematical operations.

## Benchmarks

Below are results of running various benchmarks on the GPU and CPU hardware shown in the table above. Note that benchmarks were performed on a system consisting of a dual-socket motherboard with 2 processors (8 physical cores total). We benchmark a single-GPU implementation, single CPU core implementation, and a multithreaded CPU implementation. In all cases the effective bandwidth is calculated by dividing the total bytes involved in memory access operations divided by the total execution time of a element-wise vector addition function/kernel. This is a lower bound on the true bandwidth as some time will be spent by the function/kernel performing the addition operation. A value of N=1e6 is used in all cases.

In [15]:
import pandas as pd
df = pd.read_csv("bmarks.csv")
df

,Parallel Implementation,Bandwidth (GB/s)
0,Serial,4.4
1,OpenMP - 2 cores,2.8
2,OpenMP - 4 cores,2.6
3,OpenMP - 8 cores,2.7
4,CUDA - tiled,201.0


The GPU performance blows the CPU out of the water. This is not unexpected given the theoretical bandwidth numbers shown above, however it is a powerful example of how GPUs can yield massive speedups for the simplest of memory-bound problems. Note that CUDA applications also require expensive operations in which data is moved from the host to device memory, which is not accounted for in the numbers above. This is purely a measure of bandwidth after any necessary data transfers have occured. 

Note that we are getting nowhere near the peak theoretical performance on the GPU or CPU. Part of that can be explained by our method for measuring bandwidth (we did not subtract time spent by the CPU performing addition operations). 

It's also noteworthy that CPU-based multithreading does not improve performance for this simple problem. In fact, effective bandwidth actually drops slightly with multiple threads/cores involved. This is because threads share hardware resources for accessing memory, resulting in a classic Von Neumann bottleneck scenario.